Plot the band structure, DOS/PDOS and IPA dielectric function for a database entry.

This version is optimized for comparing the LDA, QSGW, and QSGW^ results for semiconductors and insulators.
For example, if a calculation crashes during the QSGW^, the notebook will only plot the LDA and QSGW results.

In [ ]:
"""
START USER INPUT 
"""

db_name  = "../../questaal_database"
db_entry = "CuCl_icsd_78270_nsites_2.json" # examples: "Si_icsd_51688_nsites_2.json", "CuCl_icsd_78270_nsites_2.json"

bs_ylim  = [-10, 10] # energy range for the band structure plot (eV)
eps_xlim = [0, 10] # energy range for the ipa dielectric function plot (eV)
eps_ylim = [-30, 30] # axis limit for the total dielectric function (difficult to estimate due to the Drude tail)
figsize  = [3, 3] # figure size (width, height) in inches

"""
END USER INPUT 
"""

# external imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from qsgw_workflow.utils.helper import load_db_entry, plot_bs

# the cluster has no LaTeX ...
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "DejaVu Serif"

# fix the color and linestyles for all plots
lda_color_and_linestyle = "r--"
qsgw_color_and_linestyle = "k-"
qsgwbse_color_and_linestyle = "b-"

# load the files
db_path = os.path.join(db_name, db_entry)
if os.path.exists(db_path):
    print(f"Loading the following database entry: {db_path:s}")
    cse = load_db_entry(db_path)
else:
    sys.exit("The given database name or database entry does not exist!")
    
# check if the QSGW calculation finished
if cse.parameters["qsgw_flag"] == False:
    sys.exit("The QSGW calculation has not finished or has crashed!")

# is everything converged?
if cse.parameters["qsgw_kpt_conv_error_flag"] == 1:
    print("The QSGW k-grid convergence stopped at the DFT k-grid.")
if cse.parameters["qsgw_scf_conv_error_flag"] == 1:
    print("The QSGW self-consistency loop did not converge.")

# print the band gap (if possible)
def bs_gap_estimation(bs, vbm_idx):
    """
    The difference between the band gap obtained from the regular DFT k-grid
    and that obtained from the band structure calculation is typically between 1 and 10 meV.
    """
    vb = []
    cb = []
    for path in bs["bs_paths"]:
        vb.extend(path["bands"][:, vbm_idx])
        cb.extend(path["bands"][:, vbm_idx + 1])
    return np.max([0, np.min(cb) - np.max(vb)])
def band_gap(cse, label):
    if f"gap_{label.lower():s}" in cse.data.keys():
        gap = max([0, cse.data[f"gap_{label.lower():s}"]])
        corrected_gap = bs_gap_estimation(cse.data[f"bs_{label.lower():s}"], cse.parameters["vbm_idx"])
        print(f"{label.upper():7s} | 'llmf' band gap = {gap:2.2f} eV | 'band structure' band gap = {corrected_gap:2.2f} eV")
band_gap(cse, "LDA")
band_gap(cse, "QSGW")
band_gap(cse, "QSGWBSE")

# plot the LDA and QSGW band structure
fig, ax = plt.subplots(figsize=figsize)
plot_bs(ax, cse.data["bs_lda"], lcs=lda_color_and_linestyle, deco=True)
plot_bs(ax, cse.data["bs_qsgw"], lcs=qsgw_color_and_linestyle, deco=False)
if "bs_qsgwbse" in cse.data.keys():
    plot_bs(ax, cse.data["bs_qsgwbse"], lcs=qsgwbse_color_and_linestyle, deco=False)
else:
    print("The QSGW^ seems to have crashed.")
ax.set_ylim(bs_ylim)
fig.align_xlabels()
fig.tight_layout()

# plot the LDA and QSGW DOS
fig, ax = plt.subplots(figsize=figsize)
dos1 = cse.data["dos_lda"]
energy1 = np.array(dos1["energy"])
idx = (energy1 > bs_ylim[0] + 0.1) & (energy1 < bs_ylim[1])
tdos1 = np.array(dos1["tdos"])
ax.fill_between(energy1[idx], tdos1[idx], 0, color=lda_color_and_linestyle.split("-")[0], zorder=1, alpha=0.2, lw=0)
ax.plot(energy1[idx], tdos1[idx], lda_color_and_linestyle, zorder=2, label="LDA")
dos2 = cse.data["dos_qsgw"]
energy2 = np.array(dos2["energy"])
idx = (energy2 > bs_ylim[0] + 0.1) & (energy2 < bs_ylim[1])
tdos2 = np.array(dos2["tdos"])
ax.fill_between(energy2[idx], tdos2[idx], 0, color=qsgw_color_and_linestyle.split("-")[0], zorder=1, alpha=0.2, lw=0)
ax.plot(energy2[idx], tdos2[idx], qsgw_color_and_linestyle, zorder=2, label="QSGW")
if "dos_qsgwbse" in cse.data.keys():
    dos3 = cse.data["dos_qsgwbse"]
    energy3 = np.array(dos3["energy"])
    idx = (energy3 > bs_ylim[0] + 0.1) & (energy3 < bs_ylim[1])
    tdos3 = np.array(dos3["tdos"])
    ax.fill_between(energy3[idx], tdos3[idx], 0, color=qsgwbse_color_and_linestyle.split("-")[0], zorder=1, alpha=0.2, lw=0)
    ax.plot(energy3[idx], tdos3[idx], qsgwbse_color_and_linestyle, zorder=2, label=r"QSG$\hat{\mathrm{W}}$")
ax.axvline(x=0, color="k", linestyle="--", lw=0.5, zorder=-2)
ax.set_xlim(bs_ylim) # use the same energy range as for the band structure
ax.set_ylim(ymin=0)
ax.legend(loc="upper left", handlelength=1.75)
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("DOS (states/eV)")
fig.tight_layout()

# plot the LDA and QSGW IPA dielectric function    
fig, ax = plt.subplots(figsize=figsize)
eps1 = cse.data["eps_lda"]
omega = np.array(eps1["omega"])
eps_inter_imag = np.mean(eps1["eps_imag"], axis=1)
idx = (omega > eps_xlim[0] + 0.1) & (omega < eps_xlim[1])
ax.plot(omega[idx], eps_inter_imag[idx], lda_color_and_linestyle, label="LDA")
ymax = np.max(eps_inter_imag[idx])
eps2 = cse.data["eps_qsgw"]
omega = np.array(eps2["omega"])
eps_inter_imag = np.mean(eps2["eps_imag"], axis=1)
idx = (omega > eps_xlim[0]) & (omega < eps_xlim[1])
ax.plot(omega[idx], eps_inter_imag[idx], qsgw_color_and_linestyle, label="QSGW")
if "eps_qsgwbse" in cse.data.keys():
    eps3 = cse.data["eps_qsgwbse"]
    omega = np.array(eps3["omega"])
    eps_inter_imag = np.mean(eps3["eps_imag"], axis=1)
    idx = (omega > eps_xlim[0]) & (omega < eps_xlim[1])
    ax.plot(omega[idx], eps_inter_imag[idx], qsgwbse_color_and_linestyle, label=r"QSG$\hat{\mathrm{W}}$")
if np.max(eps_inter_imag[idx]) > ymax:
    ymax = np.max(eps_inter_imag[idx])
ax.set_xlim(eps_xlim)
ax.set_ylim(ymax=1.05*ymax)
ax.legend(loc="upper left", handlelength=1.75)
ax.set_xlabel(r"$\omega$ (eV)")
ax.set_ylabel(r"$\mathrm{Im}[\varepsilon$]")
fig.align_xlabels()
fig.tight_layout()